In [2]:
import torch
from torch import  nn  
from torch.nn import functional as f
net= nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X=torch.rand(2, 20)
net(X)

tensor([[-0.2201, -0.0080,  0.1533,  0.1707,  0.1488,  0.0648, -0.0206, -0.0045,
         -0.2350, -0.0501],
        [-0.2178, -0.1065,  0.1223,  0.0252,  0.1448,  0.0330,  0.0141, -0.0005,
         -0.1843, -0.1521]], grad_fn=<AddmmBackward0>)

### 自定义块

包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [3]:
from turtle import forward


class MLP(nn.Module):
    #用模型参数声明层，
    def __init__(self) :
        #调用MLP的父类Module的构造函数来执行初始化
        #在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden=nn.Linear(20, 256) #隐藏层
        self.out=nn.Linear(256, 10) #输出层

    #定义模型的前向传播，根据输入x返回所需的模型输出
    def forward(self, X):
        return self.out(f.relu(self.hidden(X)))


In [4]:
net=MLP()
net(X)

tensor([[-0.0740, -0.0989,  0.2282,  0.1443, -0.0313, -0.0626,  0.3939,  0.1684,
         -0.0522,  0.3163],
        [-0.1072, -0.0311,  0.0524,  0.1144, -0.0070, -0.0369,  0.3467,  0.1650,
          0.0021,  0.1517]], grad_fn=<AddmmBackward0>)

### 顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)]=module
    
    def forward(self, X):
        for block in self._modules.values():
            X=block(X)
        return X


In [6]:
net =MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0088,  0.2782,  0.0117,  0.0699,  0.0577,  0.1505, -0.0224,  0.0461,
          0.1404, -0.0487],
        [-0.0034,  0.2448, -0.1334,  0.0813,  0.0863,  0.0595,  0.0348, -0.0016,
          0.0833, -0.0226]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = f.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net= FixedHiddenMLP()
net(X)

tensor(-0.0021, grad_fn=<SumBackward0>)

### 组合块

In [9]:
class NesMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                            nn.Linear(64, 32), nn.ReLU())
        self.linear=nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
chimera =nn.Sequential(NesMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)






tensor(-0.3418, grad_fn=<SumBackward0>)

### 参数管理
* 访问参数，用于调试、诊断和可视化。
* 参数初始化。
* 在不同模型组件间共享参数。

In [10]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X=torch.rand(size=( 3, 4))
net(X)


tensor([[0.5036],
        [0.3327],
        [0.3758]], grad_fn=<AddmmBackward0>)

#### 访问参数

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1446,  0.0336,  0.0415, -0.1445,  0.0628,  0.1722,  0.1415,  0.2832]])), ('bias', tensor([0.2138]))])


#### 目标参数
每个参数都表示为参数类的一个实例。 要对参数执行任何操作，首先我们需要访问底层的数值。 有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2138], requires_grad=True)
tensor([0.2138])


In [13]:
net[2].weight.grad==None

True

#### 一次性访问所有参数

In [14]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [23]:
net.state_dict()['2.bias'].data

tensor([0.2138])

#### 从嵌套块收集参数

In [30]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                                     nn.Linear(8, 4), nn.ReLU())
def block2():
    net= nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

rgent=nn.Sequential(block2(), nn.Linear(4, 1))
rgent(X)

tensor([[0.5478],
        [0.5478],
        [0.5480]], grad_fn=<AddmmBackward0>)

In [31]:

print(rgent)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [32]:
rgent[0][1][0].bias.data

tensor([ 0.1070,  0.3457, -0.1451,  0.2746, -0.3987,  0.1669, -0.3618,  0.2244])

### 参数初始化
默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。 PyTorch的nn.init模块提供了多种预置初始化方法。
#调用内置的初始化器，下面将所有权重参数初始化为标准差为0.01的高斯随机变量

In [33]:
#调用内置的初始化器，下面将所有权重参数初始化为标准差为0.01的高斯随机变量
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0064, -0.0032,  0.0010,  0.0221]), tensor(0.))

In [36]:
# 将所有参数初始化为给定的常数，比如初始化为1
def init_constant(m):
    if type(m) ==nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [37]:
#自定义初始化
from unicodedata import name


def my_init(m):
    if type(m)==nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()] [0])
        nn.init.uniform_(m.weight, -10, -10)
        m.weight.data *=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-10., -10., -10., -10.],
        [-10., -10., -10., -10.]], grad_fn=<SliceBackward0>)

### 参数绑定
在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数

In [38]:
shared=nn.Linear(8, 8)

net= nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                                shared, nn.ReLU(),
                                shared,  nn.ReLU(),
                                nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0, 0]=100

print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
